In [29]:
def kummer_schematic_addition(xP, xQ, A=0):
    """
    À partir de x(P) et x(Q), retourne le polynôme quadratique
    ayant pour racines x(P+Q) et x(P−Q).
    Supposé être sur une courbe de Montgomery : By^2 = x(x^2 + Ax + 1)

    Entrées :
        - xP, xQ : abscisses des points P et Q (dans un corps K)
        - A : paramètre du modèle de Montgomery (par défaut 0)

    Sortie :
        - polynôme f(X) = X^2 - S·X + P, avec racines x(P+Q), x(P−Q)
    """
    K = xP.parent()
    R.<X> = PolynomialRing(K)

    # Formules classiques pour les courbes de Montgomery (cf. Costello–Smith)
    S = (xP * xQ - 1)^2
    D = (xP - xQ)^2

    # Produit = x(P+Q)·x(P−Q)
    product = S / D

    # Somme = x(P+Q) + x(P−Q) = 2A + (xP + xQ)(xP xQ - 1) / (xP - xQ)
    numerator = (xP + xQ)*(xP*xQ - 1)
    denominator = xP - xQ
    trace = 2*A + numerator / denominator

    # Polynôme dont les racines sont x(P+Q), x(P−Q)
    f = X^2 - trace*X + product
    return f


In [30]:
K = GF(101)
xP = K(5)
xQ = K(9)
A = K(2)

f = kummer_schematic_addition(xP, xQ, A)
print("Polynôme quadratique :", f)
print("Racines :", f.roots())


Polynôme quadratique : X^2 + 49*X + 20
Racines : []


In [31]:
# Exemple : courbe de Montgomery y^2 = x(x^2 + Ax + 1)
K = GF(101)
A = K(2)
B = K(1)
E = EllipticCurve(K, [0, B, 0, A, 0])  # Cette équation est by^2 = x(x^2 + Ax + 1)



In [32]:
P = E.random_point()
Q = E.random_point()


In [33]:
def direct_add_poly(P, Q):
    """
    Retourne le polynôme (X - x(P+Q))(X - x(P-Q)) à partir des points.
    """
    K = P.curve().base_field()
    R.<X> = PolynomialRing(K)
    x1 = (P + Q)[0]
    x2 = (P - Q)[0]
    return (X - x1)*(X - x2)



In [34]:
def kummer_schematic_addition(xP, xQ, A):
    K = xP.parent()
    R.<X> = PolynomialRing(K)

    numerator = (xP + xQ)*(xP*xQ - 1)
    denominator = xP - xQ
    trace = 2*A + numerator / denominator
    product = ((xP*xQ - 1)^2) / ((xP - xQ)^2)
    
    return X^2 - trace*X + product



In [35]:
import time

def compare_algos(E):
    P = E.random_point()
    Q = E.random_point()
    
    while P == -Q or P == Q or P == E(0):  # pour éviter les cas pathologiques
        P = E.random_point()
        Q = E.random_point()

    # Mesure temps méthode directe
    t1 = time.time()
    f1 = direct_add_poly(P, Q)
    t2 = time.time()

    # Mesure temps méthode Kummer
    t3 = time.time()
    f2 = kummer_poly_from_points(P, Q)
    t4 = time.time()

    print("🔹 Méthode directe (avec P, Q) :", f1)
    print("⏱️ Temps :", round((t2 - t1)*1000, 3), "ms")
    
    print("🔸 Méthode Kummer (xP, xQ)     :", f2)
    print("⏱️ Temps :", round((t4 - t3)*1000, 3), "ms")

    # Vérification égalité
    same = f1 == f2
    print("✅ Polynômes égaux :", same)

In [36]:
compare_algos(E)


🔹 Méthode directe (avec P, Q) : X^2 + 48*X + 88
⏱️ Temps : 0.235 ms
🔸 Méthode Kummer (xP, xQ)     : X^2 + 58*X + 65
⏱️ Temps : 0.084 ms
✅ Polynômes égaux : False


In [37]:

P = E.random_point()
Q = E.random_point()
xP = P[0]
xQ = Q[0]

f1 = direct_add_poly(P, Q)
f2 = kummer_schematic_addition(xP, xQ, A)

print(f"Polynôme direct : {f1}")
print(f"Polynôme Kummer : {f2}")

print("Polynômes égaux ?", f1 == f2)
print("Racines direct :", f1.roots(multiplicities=False))
print("Racines Kummer :", f2.roots(multiplicities=False))


Polynôme direct : X^2 + 98*X + 33
Polynôme Kummer : X^2 + 93*X + 58
Polynômes égaux ? False
Racines direct : [86, 18]
Racines Kummer : []
